In [1]:
import numpy as np
import esmvalcore.experimental as esmvaltool
import netCDF4 as nc
import warnings
import os
from ruamel.yaml import YAML
import yaml
import logging

warnings.filterwarnings("ignore", category=UserWarning, module='esmvalcore')


/sw/spack-levante/mambaforge-22.9.0-2-Linux-x86_64-ze6txd/lib/python3.10/site-packages/esmvalcore/experimental/_warnings.py:13: UserWarning: 
  Thank you for trying out the new ESMValCore API.
  Note that this API is experimental and may be subject to change.
  More info: https://github.com/ESMValGroup/ESMValCore/issues/498


## Get HBVmountain recipe

In [2]:
recipe = esmvaltool.get_recipe('~/ESMValTool/esmvaltool/recipes/hydrology/recipe_newHBVmountain.yml')

## Functions

In [3]:
def select_cmip6_models(str_path_to_models, model_list=None, ignore_models=None, select_all=False):
    #Hier moet commentaar over wat deze functie in doet
    model_file = yaml.safe_load(open(str_path_to_models))
            
    if ignore_models != None:
        selected_CMIP6_models = dict((k, model_file[k]) for k in model_file
           if k not in ignore_models)
        selected_CMIP6_models = tuple(selected_CMIP6_models.values()) 
    if select_all == True and model_list == None:
        selected_CMIP6_models = dict((k, model_file[k]) for k in model_file.keys()
               if k in model_file)
        selected_CMIP6_models = tuple(selected_CMIP6_models.values()) 
    if model_list != None:
        selected_CMIP6_models = dict((k, model_file[k]) for k in model_list
               if k in model_file)
        selected_CMIP6_models = tuple(selected_CMIP6_models.values())
    return selected_CMIP6_models

In [5]:
def run_recipe_HBVmountain(recipe, catchment_name, scenario, CMIP6_models, start_year, end_year, catchment_id=None):
    recipe.data["preprocessors"]["daily"]["extract_shape"]["shapefile"] = os.path.join(catchment_name, catchment_name + '.' + 'shp') #shape
    recipe.data["diagnostics"]["diagnostic_daily"]["scripts"]["script"][
        "basin"
    ] = catchment_name #basin
    
    recipe.data["diagnostics"]["diagnostic_daily"]["additional_datasets"] = [
    *CMIP6_models
]
    variables = recipe.data["diagnostics"]["diagnostic_daily"]["variables"]
    var_names = "tas", "pr"
    
    for var_name in var_names:
        variables[var_name]["exp"] = scenario
        
        
    startyear = start_year #get_time(start_time).year
    for var_name in var_names:
        variables[var_name]["start_year"] = startyear

    endyear = end_year # get_time(end_time).year
    for var_name in var_names:
        variables[var_name]["end_year"] = endyear
    recipe_output = recipe.run()
    print(f'recipe stored in {os.path.dirname(str(list(recipe_output.values())[0].files[0].path))}')
    return recipe_output

## Select CMIP6 models

In [6]:
CMIP6_models = select_cmip6_models('climate_models.yml', ignore_models=['AWI-CM-1-1-MR','HadGEM3-GC31-LL', 'KACE-1-0-G'])
CMIP6_models = select_cmip6_models('climate_models.yml', model_list=['AWI-CM-1-1-MR','HadGEM3-GC31-LL', 'KACE-1-0-G'])


## Run recipe

In [7]:
recipe_output = run_recipe_HBVmountain(recipe, catchment_name='Conasauga', scenario='historical',
                                       CMIP6_models=CMIP6_models, start_year=1975, end_year=1978, catchment_id=None)

ERROR:esmvalcore._recipe_checks:No input files found for variable {'mip': 'day', 'project': 'CMIP6', 'exp': 'historical', 'preprocessor': 'daily', 'variable_group': 'pr', 'short_name': 'pr', 'diagnostic': 'diagnostic_daily', 'dataset': 'AWI-CM-1-1-MR', 'ensemble': 'r1i1p1f1', 'grid': 'gn', 'recipe_dataset_index': 0, 'timerange': '1975/1978', 'institute': ['AWI'], 'activity': 'CMIP', 'alias': 'AWI-CM-1-1-MR', 'original_short_name': 'pr', 'standard_name': 'precipitation_flux', 'long_name': 'Precipitation', 'units': 'kg m-2 s-1', 'modeling_realm': ['atmos'], 'frequency': 'day', 'start_year': 1975, 'end_year': 1978}
ERROR:esmvalcore._recipe_checks:Looked for files matching ['pr_day_AWI-CM-1-1-MR_historical_r1i1p1f1_gn*.nc'], but did not find any existing input directory
ERROR:esmvalcore._recipe_checks:Set 'log_level' to 'debug' to get more information
ERROR:esmvalcore._recipe:Could not create all tasks
ERROR:esmvalcore._recipe:Missing data for preprocessor diagnostic_daily/pr:
- Missing da

RecipeError: Could not create all tasks